In [1]:
#@markdown **You need to sign up for [DagsHub](https://dagshub.com/user/sign_up) , then enter the name of the repository you'd like to create, and your username and email.**

#@markdown Enter the repository name for the project:
REPO_NAME= "komrag" #@param {type:"string"}

#@markdown Enter the username of your DAGsHub account:
USER_NAME = "OsipovStas" #@param {type:"string"}

#@markdown Enter the email for your DAGsHub account:
EMAIL = "stasstels@gmail.com" #@param {type:"string"}

#@markdown ---

import IPython

In [2]:
from dotenv import load_dotenv

load_dotenv("../.env")

In [3]:
import os

DH_TOKEN = os.environ['LLM_MATH_COMP_DH_TOKEN']

In [4]:
import mlflow
import dagshub
import os
#DH_TOKEN = os.environ['LLM_MATH_COMP_DH_TOKEN']
username = f'{USER_NAME}'  # Replace with your DagsHub username
repository = f'{REPO_NAME}'  # Replace with your repository name
token = f'{DH_TOKEN}'  # Replace with your DagsHub token
os.environ['MLFLOW_TRACKING_USERNAME'] = username
os.environ['MLFLOW_TRACKING_PASSWORD'] = token
dagshub.auth.add_app_token(token)
dagshub.init(repo_name=REPO_NAME, repo_owner=USER_NAME)
mlflow.set_tracking_uri(f"https://dagshub.com/{USER_NAME}/{REPO_NAME}.mlflow")

In [5]:
from dotenv import load_dotenv

load_dotenv("../.env")

In [6]:
from dotenv import load_dotenv

load_dotenv("../.env", override=True)

In [7]:
from dotenv import load_dotenv

load_dotenv("../.env", override=True)

In [8]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("OsipovStas/komrag")


s3.download_file(
    Bucket="komrag",  # name of the repo
    Key="life_begin.pdf",  #  remote path from where to download the file
    Filename="life_begin_copy.pdf",  # local path where to download the file
)

In [9]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model=model, temperature=0, timeout=120)

In [10]:
model="gpt-4o-mini-2024-07-18"

In [11]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model=model, temperature=0, timeout=120)

In [12]:
from langchain_openai import AzureChatOpenAI

chat = AzureChatOpenAI(model=model, temperature=0, timeout=120)

In [13]:
chat.invoke("Hello, do you know russian? Write a response in it")

In [14]:
from langchain_openai import AzureEmbeddingsOpenAI

In [15]:
from langchain_openai import AzureOpenAIEmbeddings

In [16]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(model="text-embedding-3-small-1", dimensions=768)

In [17]:
len(embeddings.embed_query("Приветы, как дела?"))

In [18]:
file_path = "life_begin_copy.pdf"

In [19]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [20]:
len(pages)

In [21]:
template="""
You will receive a text in Russian extracted using an OCR tool.
Your task is to clean and format this text to enhance readability.
The text may contain various artifacts introduced during the OCR process, such as:

 - Special characters or symbols that do not belong.
 - Inconsistent casing of letters (e.g., random uppercase or lowercase letters).
 - Incorrect spacing within words or between words (e.g., words glued together or unnecessary spaces).
 - Hyphenation errors, such as words split across lines.
 - Incorrect punctuation or misplaced commas and periods.
 - New line characters

Please ensure the final output is free from these artifacts and is well-formatted.
Pay attention to punctuation, paragraph structure, and overall coherence.
The goal is to produce a clean, easy-to-read text.
Output ONLY the cleaned text.


### TEXT ###

{text}

"""

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)

In [23]:
chain = prompt | chat | StrOutputParser()

In [24]:
print(chain.invoke({"text": pages[30].page_content}))

In [25]:
def clean_text(text):
    # try up to 3 times if no success return text as is
    for _ in range(3):
        try:
            return (chain.invoke({"text": text}), True)
        except Exception as e:
            print(f"Error cleaning text: {e}")
    return (text, False)

In [26]:
from tqdm import tqdm

failures = []
for p in tqdm(pages, desc="Cleaning text", unit="page"):
  text, suc = clean_text(p.page_content)
  if not suc:
    failures.append(p)
  else:
    p.page_content = text

In [27]:
print(pages[3].page_content)

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
splits = text_splitter.split_documents(pages)

len(splits)

In [29]:
splits[8]

In [30]:
import pinecone

pc = pinecone.Pinecone(api_key=userdata.get('PINECONE_KEY'))

In [31]:
from dotenv import load_dotenv

load_dotenv("../.env", override=True)

In [32]:
import pinecone

pc = pinecone.Pinecone(api_key=os.environ['PINECONE_API_KEY'])

In [33]:
pc.list_indexes()

In [34]:
import time

index_name = "komrag"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [35]:
from pinecone import Pinecone, ServerlessSpec

pc = pinecone.Pinecone(api_key=os.environ['PINECONE_API_KEY'])

In [36]:
pc.list_indexes()

In [37]:
import time

index_name = "komrag"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [38]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [39]:
import hashlib

def generate_id(content):
    # Ensure the content is in bytes
    if isinstance(content, str):
        content = content.encode('utf-8')

    # Create a SHA-256 hash object
    hash_object = hashlib.sha256()

    # Update the hash object with the content
    hash_object.update(content)

    # Get the hexadecimal representation of the hash
    unique_id = hash_object.hexdigest()

    return unique_id

In [40]:
def index_all_docs(docs):
  ids = [generate_id(doc.page_content) for doc in docs]
  vector_store.add_documents(documents=docs, ids=ids)

In [41]:
pc.list_indexes()

In [42]:
index_all_docs(pages)